## Introduction
std::thread is implemented using the system's native thread implementation. Depending on the system some functionality might not be available in standard C++, then we have to use the system's thread implementation directly. Things like thread priority - give a thread a higher or a lower share of precossor time and thread affinity - pin the thread on a specific processor core(to remove context switch time), might not be available in standard C++ depending on the system. Each thread has a handle that is used internally by the system's thread native implementation, this handle is needed to make calls into the native implementation's API. We can get this handle from native_handle() member function.
```
void hello() 
{
    std::cout << "Hello, thread!" << std::endl;
}

std::thread thr(hello);

//Display the threads native handle
std::cout << "Hello thread has a native handle : " << thr.native_handle() << std::endl;

thr.join();
```
Each executing thread has an identifier, guaranteed to be unique.
```
//Return the identifier of the current thread
std::this_thread().get_id();

//Return the identifier associated with a std::thread object
thr.get_id();
```
We can pause a thread or make it sleep using sleep_for() which takes argument of type std::chrono::duration.
```
std::this_thread()::sleep_for(2s); //or std::this_thread()::sleep_for(std::chrono::seconds(2));
```

## C++ thread class
std::thread class is implemented using RAII similar to std::unique_ptr and std::fstream, constructor acquires the resource and destructor releases it. We cannot copy std::thread object, as only one thread object can be bound to an executing thread at a time. This means std::thread is a move-only class. Exceptions in threads are handled similar to single threaded application, each thread has its own execution stack. This stack is unwound when the thread throws exception, destructors for all local objects in scope are called, the program moves up the thread's stack until it finds a suitable exception handler, if no handler is found the program is terminated. With the std::thread class there is no way for other threads to catch the exception.
```
void hello()
{
    throw std::exception();
    std::cout << "Hello, thread!" << std::endl;
}

//Exception will not be caught, as catching is on a different thread
//Program will be terminated, since exception is not handled
try
{
    std::thread thr(hello);
    thr.join();
}
catch(std::exception &e)
{
    std::cout << "Exception caught: " << e.what() << std::endl;
}
```
Instead of join(), we can call detach(). In that case the parent will continue executing and the two threads will run completely independent. The child thread can run even after the parent is terminated. Eventually when the child thread's destructor is called it will not terminate the program as it is detached.  
When an exception is thrown in parent thread, then destructors are called for every object in scope, including the child std::thread's destructor. std::thread's destructor checks whether join() or detach() has been called, if neither, it calls std::terminate(). We must make sure join() or detach() is called in every path of the code.
```
void hello()
{
    std::cout << "Hello, thread!" << std::endl;
}

//Program will be terminated, when the exception is thrown
//when the exception is thrown, std::thread's destructor is called
//Since no join() or detach() is called yet, program will terminate
try
{
    std::thread thr(hello);
    
    throw std::exception();
    
    thr.join();
}
catch(std::exception &e)
{
    std::cout << "Exception caught: " << e.what() << std::endl;
}
```
One possible solution is to also call join() in the catch block. A better solution would be to use RAII idom, where we will have a class that has the std::thread object as its member, and the destructor of this class will call the join(). There is also a joinable() member function, returns false if join() or detach() is already called or if the thread has completed execution.
```
class thread_guard
{
public:
    //Constructor takes rvalue reference argument, std::thread is move-only
    explicit thread_gaurd(std::thread &&thr) : m_thr(std::move(thr))
    {
    }
    
    //Destructor, join the thread is necessary
    ~thread_guard()
    {
        if(m_thr.joinable())
        {
            m_thr.join();
        }
    }
    
    thread_guard(const thread_guard&) = delete;
    thread_guard& operator=(const thread_guard&) = delete;
    
private:
    std::thread m_thr;
};
```
Some native threads can be killed, suspend, cancelled or terminated. In general, abruptly terminating a thread is not a good idea. std::thread does not support this. C++ 20 has std::jthread(join thread), which calls join in its destructor if necessary. std::jthread supports cooperative interruption, so things like suspend are possible with std::jthread. 

## Multiple threads
All the threads in a program share the same memory space, so to share the data, data should just be visible to these thread functions. As threads interleave their execution and can interfere with each other actions, modifying shared data can cause data corruption.  
A data race occurs when two or more threads access the same memory location, at least one of the threads modifies it, potentially conflicting access to the same memory location. If the threads are syncronized, there will be no data race and data corruption. Syncronization makes sure that only one thread accesses the shared data at a given time.  
Race condition is a situation where the outcome depends on when threads are scheduled to run. Data race is a special case of a race condition. For example when one client is clearing a database table, another client inserts an entry into the same table. Result of this depends on which client executes first and if they execute interleaved.
```
void print(std::string str)
{
    for(int i = 0; i < 5; i++)
    {
        std::cout << str[0] << str[1] << str[2] << std::endl;
    }
}

int main()
{
    std::thread thr1(print, "abc");
    std::thread thr2(print, "def");
    std::thread thr3(print, "xyz");
    
    thr1.join();
    thr2.join();
    thr3.join();
}
```
In this program, the output will be interleaved, as the scehdular can schedule any thread at a given point. Data race can cause incorrect results, incorrect program flow, tron writes/reads and objects which are improperly constructed/destroyed. We can prevent data race's from not occuring by not sharing data between threads, if that is not possible syncronize the threads. Syncronization has significant costs, increases the execute time and makes the program more complex.